This file is just for testing. This file will create symbols.csv which get_data.py will use to pull data. 

In [3]:
import pandas as pd, numpy as np, json, requests, time

In [11]:
def get_all_products():
    """
    Reference: https://docs.cloud.coinbase.com/exchange/reference/exchangerestapi_getproducts

    Returns:
        DataFrame: metadata on all known trading pairs on Coinbase
    """
    url = 'https://api.exchange.coinbase.com/products'
    headers = {'Accept': 'application/json'}
    time.sleep(0.15) # Rate limit is 10 requests per second
    response = requests.get(url, headers=headers)
    j = json.loads(response.text)
    df = pd.DataFrame(j)
    return df


def get_product_stats(symbol):
    """
    Formats API call to Coinbase to pull 24 hour stats on a symbol
    Reference: https://docs.cloud.coinbase.com/exchange/reference/exchangerestapi_getproductstats
    `volume` is in base currency units. `open`, `high`, `low` are in quote currency units.

    Args:
        symbol (str): ticker symbol to lookup

    Returns:
        dict: includes `open`, `high`, `low`, `last`, `volume`, `volume_30day`            
    """
    url = 'https://api.exchange.coinbase.com/products/' + symbol + '/stats'
    headers = {'Accept': 'application/json'}
    time.sleep(0.15) # Rate limit is 10 requests per second
    response = requests.request('GET', url, headers=headers)
    j = json.loads(response.text)
    j['symbol'] = symbol
    return j

In [12]:
# Get USD products on Coinbase
all_products = get_all_products()
usd_products = all_products[all_products['quote_currency'] == 'USD']

# pull daily stats for USD products
usd_product_stats = []
for symbol in usd_products['id']: 
    usd_product_stats.append(get_product_stats(symbol))
usd_product_stats = pd.DataFrame(usd_product_stats).sort_values('volume_30day', ascending=False)
usd_product_stats = usd_product_stats.drop('message', axis=1).dropna() # Drop XRP

# convert volume to quote currency (USD)
usd_product_stats['volume_in_USD'] = usd_product_stats['volume'].astype(float) * usd_product_stats['open'].astype(float)
usd_product_stats['volume_30day_in_USD'] = usd_product_stats['volume_30day'].astype(float) * usd_product_stats['open'].astype(float)

# Sort and write to file
usd_product_stats = usd_product_stats.sort_values('volume_30day_in_USD', ascending=False)
# usd_product_stats.to_csv('daily_stats.csv', index=False)

In [39]:
# Create ../data/symbols.csv
symbols_to_include = usd_product_stats['symbol'].iloc[0:20]
pd.DataFrame(symbols_to_include).to_csv('../data/symbols.csv', index=False)